<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/NBlemma_Scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lemmatization

## Importing Data

In [1]:
# mount Google Drive
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [2]:
# import file from Google Drive
import pandas as pd
df = pd.read_csv('/drive/My Drive/book32listing.csv',encoding='latin1', header=None)

In [11]:
# drop columns that are not needed
#df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6,5]] # only columns with titles and genres
df1.columns = ['title', 'genre', 'label']
print(df1)

                                                    title      genre  label
0                         Mom's Family Wall Calendar 2016  Calendars      3
1                         Doug the Pug 2016 Wall Calendar  Calendars      3
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars      3
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars      3
4                    Sierra Club Engagement Calendar 2016  Calendars      3
...                                                   ...        ...    ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel     29
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel     29
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel     29
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel     29
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel     29

[207572 rows x 3 columns]


## Preprocessing

In [18]:
# case collapsing
df1['title'] = df1.title.map(lambda x: x.lower())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
# remove punctuation
df1['title'] = df1.title.str.replace('[^\w\s]', '')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# import tokenizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [21]:
# tokenization
df1['title'] = df1['title'].apply(nltk.word_tokenize)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
# lemmatization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
 
df1['title'] = df1['title'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
# transform data into occurrences
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df1['title'] = df1['title'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df1['title'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [24]:
# tf-idf
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

## Training NB Model

In [26]:
# split data into train (80%) and test (20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df1['label'], test_size=0.2, random_state=69)

In [27]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

## Evaluating NB Model

In [28]:
# accuracy
import numpy as np

predicted = model.predict(X_test)
print(np.mean(predicted == y_test))

0.5445501625918343


In [30]:
# compute overall accuracy, precision, recall, f1 scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Accuracy: ', accuracy_score(y_test, predicted))
print('Precision: ', precision_score(y_test, predicted, average='weighted', zero_division=1))
print('Recall: ', recall_score(y_test, predicted, average='weighted', zero_division=1))
print('F1:', f1_score(y_test, predicted, average='weighted'))

Accuracy:  0.5445501625918343
Precision:  0.6477135302959284
Recall:  0.5445501625918343
F1: 0.5060259538699764


In [31]:
# compute accuracy, precision, recall, f1 scores by genre

from sklearn.metrics import precision_recall_fscore_support as score

# precision, recall, fscore, support separated by genre
precision, recall, fscore, support = score(y_test, predicted)

df_acc = pd.DataFrame()
df_acc['precision']=pd.Series(precision)
df_acc['recall']=pd.Series(recall)
df_acc['fscore']=pd.Series(fscore)
df_acc['support']=pd.Series(support)

index = list(y_test.unique())
df_acc.index = index

print(df_acc)

    precision    recall    fscore  support
19   0.747807  0.260703  0.386621     1308
7    0.645833  0.037037  0.070056      837
9    0.524390  0.756281  0.619342     1990
23   0.912281  0.728000  0.809789      500
2    0.336946  0.771533  0.469048     2740
27   0.960000  0.160535  0.275072      598
16   0.821053  0.723562  0.769231     1617
21   0.828874  0.766839  0.796651     1737
4    0.627721  0.723502  0.672217     1953
11   0.527495  0.709200  0.604999     1826
8    0.964286  0.111340  0.199630      485
28   0.424832  0.765123  0.546321     2397
20   0.591859  0.400298  0.477585     1344
29   0.699045  0.325909  0.444557     1347
15   0.771304  0.607534  0.679693     1460
12   0.525826  0.329450  0.405094     1545
13   0.604580  0.796781  0.687500     2485
17   0.928571  0.030303  0.058691      429
14   1.000000  0.009804  0.019417      510
1    0.947368  0.028391  0.055130      634
22   0.770115  0.101515  0.179384      660
10   0.681655  0.514946  0.586687     1472
30   0.8882

In [29]:
# confusion matrix
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[ 341    0   61 ...  258    0    0]
 [   4   31   25 ...  257    0    0]
 [   5    0 1505 ...   92    0    0]
 ...
 [   3    0   22 ... 3419    0    0]
 [   3    0   12 ...   57    1    0]
 [   0    0   89 ...   13    0    5]]


In [ ]:
# examine class distribution
y_test.value_counts()